In [1]:
import os
import numpy as np
import scipy as sp
import sklearn as sk
import pandas as pd
import operator
from operator import sub
from decimal import Decimal
from sklearn import preprocessing
import statistics
import seaborn as sb
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pybedtools
from pybedtools import BedTool

# This notebook is dedicated to finding the dominant single bp TSS for each gene

# Make bed file

In [9]:
# read in the file produced by CAGEr with gene names associated

df = pd.read_csv('181130single_tss_per_gene.csv')


# convert the file to a bed file

tlbed = pd.DataFrame(columns=['chr', 'start', 'end', 'name', 'score', 'strand'])

tlbed['chr'] = df['coordinates'].str.split(':', expand=True)[0]
tlbed['start'] = (df['coordinates']
                      .str.split(':', expand=True)[1]
                      .str.split('-', expand=True)[0]) 
tlbed['start'] = tlbed['start'].astype(int)
tlbed['end'] = (df['coordinates']
                    .str.split(':', expand=True)[1]
                    .str.split('-', expand=True)[1])
tlbed['name'] = df['genes']
tlbed['score'] = 1
tlbed['strand'] = df['coordinates'].str.split(':', expand=True)[2]

# fill in the end value for single bp peaks and then add one to all
tlbed['end'].fillna(value=tlbed['start'], inplace=True)
tlbed['end'] = tlbed['end'].astype(int) + 1

tlbed.to_csv("bedfiles/181130single_tss_per_gene.bed", sep="\t", header=None, index=False)

# Read in the data for each individual experiment, convert to bed

In [3]:

# convert the data for each sample into a bed file

# define function

def convertBed(x):
    x_df = pd.read_csv('TLseq_indiv_samples/'+ x + '_clusters.csv', sep = '\t', header = None)
    y = x_df[[0, 1, 2, 7, 8, 4]]
    y[1] = x_df[1] - 1
    y = y[y[8] > 1]
    y = y[~y[0].str.contains('chr_spike')]
    y.to_csv('TLseq_indiv_samples/' + x + '.bed', sep= '\t', index=False, header=False)
 

# run the function

convertBed('TL_2h_R1')
convertBed('TL_4h_R1')
convertBed('TL_2h_R3')
convertBed('TL_4h_R3')
convertBed('TL_2h_R5')
convertBed('TL_4h_R5')



/mnt/unallab/tresenrider/programs/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# Run bedtools intersect with designated file compared to each individual sample's peak calls

In [14]:
# define functions

def toIntersect(directory, bedA, intersect):
    # creates the intersect files
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".bed"):
            print(directory + filename)
            a = pybedtools.BedTool(bedA)
            b = pybedtools.BedTool(directory + filename)
            b.intersect(a, wo = True, s = True).saveas(directory + filename[:-3] + intersect)

    # read in the intersect files to python
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(intersect):
            print(os.path.join(directory, filename))
            x = pd.read_csv(directory + filename, sep = '\t', header = None)
            y = pd.DataFrame(columns = x.columns)
            # drop duplicates based on gene name and tpm value
            for gene, group in x.groupby(9):
                keep_index = group.iloc[:,4].idxmax()
                keep = x.loc[keep_index]
                y = y.append(keep)
            # drop unecessary columns 
            y.drop([1,2,6,10,11,12], inplace = True, axis = 1)
            # add to list of df, these will all be merged later 
            TSS_df_list.append(y)

def toTpm(intersect, bedA):   
    TSS_master = TL_2h_R5.merge(TL_2h_R3, how = 'outer', 
                                    left_on=9, 
                                    right_on=9, 
                                    suffixes = ('_2h_R5', '_2h_R3'))
    TSS_master = TSS_master.merge(TL_2h_R1, how = 'outer',
                                  left_on=9,
                                  right_on=9,
                                  suffixes = ('_2h_R3', '_2h_R1'))
    TSS_master = TSS_master.merge(TL_4h_R5, how = 'outer',
                                  left_on=9,
                                  right_on=9,
                                  suffixes = ('_2h_R1', '_4h_R5'))
    TSS_master = TSS_master.merge(TL_4h_R3, how = 'outer',
                                  left_on=9,
                                  right_on=9,
                                  suffixes = ('_4h_R5', '_4h_R3'))
    TSS_master = TSS_master.merge(TL_4h_R1, how = 'outer',
                                  left_on=9,
                                  right_on=9,
                                  suffixes = ('_4h_R3', '_4h_R1'))
    # reduced columns to only the peak calls for each sample
    TSS_select = TSS_master[[9, '3_2h_R5', '3_2h_R3', '3_2h_R1', '3_4h_R5', '3_4h_R3', '3_4h_R1']]
    
    # set index as gene name
    TSS_select.set_index(9, inplace=True)
    
    # converted strings to floats
    TSS_select.astype(float)
    
    # found the mode for each row and added it to TSS_select
    TSS_merge = TSS_select.merge(TSS_select.mode(axis=1), how = 'outer', left_index=True, right_index=True)
    
    # Merge back to the master which has the tpm values
    TSS_master.set_index(9, inplace=True)
    TSS_master = TSS_master.merge(TSS_merge, how = 'outer', left_index=True, right_index=True)
    
    # drop all rows in which there is one clear TSS, these peaks are already figured out
    # we just need to do more work with a subset
    TSS_2starts = TSS_master.dropna(subset = [1])

    # get only the tpm and bp peak columns for each sample
    searchfor = ['3_', '4_']
    TSS_2starts =  TSS_2starts.loc[:,TSS_2starts.columns.str.contains('|'.join(searchfor))== True]

    samples = ['2h_R5', '2h_R3', '2h_R1', '4h_R5', '4h_R3', '4h_R1']

    # convert to tuples so that I can associate a peak with the tpm of that cluster? or peak? I don't remember

    for i in samples:
        TSS_2starts[i] = list(zip(TSS_2starts.loc[:, '3_' + i + '_x'], TSS_2starts.loc[:, '4_' + i]))

    # drop all of the non tuple coulmns
    TSS_2starts.drop(TSS_2starts.columns.to_series()['3_2h_R5_x':'3_4h_R1_y'], axis=1, inplace=True)
    
    tuples_ = []

    for column, row in TSS_2starts.iterrows():
        #print(row)
        # find all unique values for the first value of a tuple
        my_set = {x[0] for x in row}
        # sum all of the second values if their first value is the same 
        my_sum = [(i,sum(x[1] for x in row if x[0] == i)) for i in my_set]
        # find the max value
        max_ = max(my_sum, key=lambda x:x[1])
        # add to the tuples_ list
        tuples_.append(max_)

    #add tuples_ as a column called best
    TSS_2starts['best']= [i[0] for i in tuples_]
    
    # add the best column to the TSS_merge df which has all genes
    # fill in the best column with previosuly calculated values in column 0

    TSS_2starts_best = pd.DataFrame(TSS_2starts['best'])
    TSS_merge2 = TSS_merge.merge(TSS_2starts_best, how = 'outer', left_index=True, right_index=True)
    TSS_merge2.best.fillna(TSS_merge2.loc[:,0], inplace=True)
    
    # make a mini df and save as CSV, all it contains are the genes and the most dominant TSSs
    TSS_final = TSS_merge2['best']
    TSS_final.to_csv('181130TSS_single_bp_' + intersect + '.csv', sep='\t', header=None)

    
    # save as a bed! Now it can be used with deeptools2!!!!!
    bed = pd.read_csv(bedA, sep='\t', header=None)
    bed_TSS = bed.merge(pd.DataFrame(TSS_final), how = 'outer', left_on=3, right_index=True)
    bed_TSS = bed_TSS.dropna('rows')
    bed_TSS.iloc[:,1] = (bed_TSS['best'] - 1).astype(int)
    bed_TSS.iloc[:,2] = bed_TSS['best'].astype(int)
    bed_TSS.iloc[:,4] = bed_TSS.iloc[:,4].astype(int)
    bed_TSS.to_csv('bedfiles/181130TSS_single_bp_' + intersect + '.bed', sep='\t', header=None, index=False)

    # this outputs a list with each sample as an entry
    # the order in which the files are processed should come out below and the next steps should take this into account
    
 

# For all genes

In [10]:
directory = 'TLseq_indiv_samples/'
bedA = 'bedfiles/181130single_tss_per_gene.bed'
intersect = 'all'

# creates a new list
TSS_df_list = []

toIntersect(directory, bedA, intersect)

TLseq_indiv_samples/TL_2h_R1.bed
TLseq_indiv_samples/TL_4h_R5.bed
TLseq_indiv_samples/TL_4h_R3.bed
TLseq_indiv_samples/TL_2h_R3.bed
TLseq_indiv_samples/TL_4h_R1.bed
TLseq_indiv_samples/TL_2h_R5.bed
TLseq_indiv_samples/TL_2h_R5.all
TLseq_indiv_samples/TL_4h_R1.all
TLseq_indiv_samples/TL_2h_R3.all
TLseq_indiv_samples/TL_4h_R3.all
TLseq_indiv_samples/TL_4h_R5.all
TLseq_indiv_samples/TL_2h_R1.all


In [15]:
# renamed output
# note that the order in which the files are in the list is reflected by the above output

TL_2h_R5 = TSS_df_list[0]
TL_4h_R1 = TSS_df_list[1]
TL_2h_R3 = TSS_df_list[2]
TL_4h_R3 = TSS_df_list[3]
TL_4h_R5 = TSS_df_list[4]
TL_2h_R1 = TSS_df_list[5]

toTpm(intersect, bedA)

/mnt/unallab/tresenrider/programs/conda/lib/python3.6/site-packages/ipykernel_launcher.py:110: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


# For ORF transcripts at LUTI loci

In [22]:
directory = 'TLseq_indiv_samples/'
bedA = 'bedfiles/prox_TSS.bed'
intersect = 'prox'

# creates a new list
TSS_df_list = []

toIntersect(directory, bedA, intersect)

TLseq_indiv_samples/TL_2h_R1.bed
TLseq_indiv_samples/TL_4h_R5.bed
TLseq_indiv_samples/TL_4h_R3.bed
TLseq_indiv_samples/TL_2h_R3.bed
TLseq_indiv_samples/TL_4h_R1.bed
TLseq_indiv_samples/TL_2h_R5.bed
TLseq_indiv_samples/TL_4h_R1.prox
TLseq_indiv_samples/TL_4h_R5.prox
TLseq_indiv_samples/TL_2h_R5.prox
TLseq_indiv_samples/TL_2h_R1.prox
TLseq_indiv_samples/TL_2h_R3.prox
TLseq_indiv_samples/TL_4h_R3.prox


In [23]:

TL_4h_R1 = TSS_df_list[0]
TL_4h_R5 = TSS_df_list[1]
TL_2h_R5 = TSS_df_list[2]
TL_2h_R1 = TSS_df_list[3]
TL_2h_R3 = TSS_df_list[4]
TL_4h_R3 = TSS_df_list[5]

toTpm(intersect, bedA)



/mnt/unallab/tresenrider/programs/conda/lib/python3.6/site-packages/ipykernel_launcher.py:110: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


# For LUTIs :)

In [18]:

directory = 'TLseq_indiv_samples/'
bedA = 'bedfiles/luti_TSS.bed'
intersect = 'luti'

# creates a new list
TSS_df_list = []

toIntersect(directory, bedA, intersect)



TLseq_indiv_samples/TL_2h_R1.bed
TLseq_indiv_samples/TL_4h_R5.bed
TLseq_indiv_samples/TL_4h_R3.bed
TLseq_indiv_samples/TL_2h_R3.bed
TLseq_indiv_samples/TL_4h_R1.bed
TLseq_indiv_samples/TL_2h_R5.bed
TLseq_indiv_samples/TL_2h_R5.luti
TLseq_indiv_samples/TL_2h_R1.luti
TLseq_indiv_samples/TL_4h_R1.luti
TLseq_indiv_samples/TL_4h_R5.luti
TLseq_indiv_samples/TL_4h_R3.luti
TLseq_indiv_samples/TL_2h_R3.luti


In [19]:

TL_2h_5R = TSS_df_list[0]
TL_2h_1R = TSS_df_list[1]
TL_4h_1R = TSS_df_list[2]
TL_4h_5R = TSS_df_list[3]
TL_4h_3R = TSS_df_list[4]
TL_2h_3R = TSS_df_list[5]

toTpm(intersect, bedA)



/mnt/unallab/tresenrider/programs/conda/lib/python3.6/site-packages/ipykernel_launcher.py:110: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


# Remove LUTI-containing loci 

In [24]:

all_TSS = pd.read_csv('bedfiles/181130TSS_single_bp_all.bed', sep='\t', header=None)
LUTI_TSS = pd.read_csv('bedfiles/181130TSS_single_bp_luti.bed', sep='\t', header=None)
ORF_TSS = pd.read_csv('bedfiles/181130TSS_single_bp_prox.bed', sep='\t', header=None)



In [25]:

noluti = all_TSS[~all_TSS.iloc[:,3].isin(LUTI_TSS.iloc[:,3])]
noluti_noprox = noluti[~noluti.iloc[:,3].isin(ORF_TSS.iloc[:,3])]

noluti_noprox.to_csv('bedfiles/181130TSS_single_bp_nonluti.bed', sep='\t', header=None, index=False)


